In [1]:
import os
os.chdir('..')
print(os.getcwd())

/mnt/SamsungSSD/workspace/yoshioka/rendering


In [2]:
import zipfile

load_path = 'modules/ManipNet/Code/Tensorflow/Data/Cylinder/train'
test_path = 'modules/ManipNet/Code/Tensorflow/Data/Cylinder/test'
save_path = 'modules/ManipNet/Code/Tensorflow/Data/ManipNetBIN'

# unzip
if((not os.path.isdir('modules/ManipNet/Code/Tensorflow/Data/Cylinder')) and os.path.exists('modules/ManipNet/Code/Tensorflow/Data/Cylinder.zip')):
    print("unzip files")
    with zipfile.ZipFile("modules/ManipNet/Code/Tensorflow/Data/Cylinder.zip","r") as zip_ref:
        zip_ref.extractall("modules/ManipNet/Code/Tensorflow/Data")


In [3]:
import numpy as np
import tensorflow as tf
from modules.manipnet_v1.MainNN_Hand.Main_EncoderRes3Bin import MainNN

mann = MainNN()


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
index_encoders=[
    np.arange(0, 192),
    np.arange(192, 528),
    np.arange(528, 2350),
    ]
dim_encoders=[[512], [512], [512]]
activation_encoders=[[0], [0], [0]]

mann.BuildModel(
        # For data process
        load_path, save_path, type_normalize=0,
        # For main network
        hiddenDim=0,
        activations=tf.nn.relu,
        # For encoder
        index_encoders=index_encoders,
        dim_encoders=dim_encoders,
        activation_encoders=activation_encoders,
        )

Data is Processed

--------------------
Encoder  0
Input dim  192
Hidden dim  [512]
Activations  [0]
--------------------

encoder0 MLP is built [192, 512] [0]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

--------------------
Encoder  1
Input dim  336
Hidden dim  [512]
Activations  [0]
--------------------
encoder1 MLP is built [336, 512] [0]
--------------------
Encoder  2
Input dim  1822
Hidden dim  [512]
Activations  [0]
--------------------
encoder2 MLP is built [1822, 512] [0]
activation <function relu at 0x7f960b268320>
MLP0 MLP is built [1536, 1536] [0]
MLP1 MLP is built [1536, 1536] [0]
MLP2 MLP is built [1536, 1536] [0]
MLP3 MLP is built [1536, 1536] [0]
endMLP MLP is built [1536, 214] [0]



In [5]:
import numpy as np
from pathlib import Path

bindata = {}
for fp in sorted(Path("modules/ManipNet/Code/Tensorflow/Data/ManipNetBIN").iterdir()):
    # print(fp)
    bindata[fp.stem] = np.fromfile(fp)

In [14]:
# print(mann.encoders[0].__dict__)
for i in range(2):
    print(f"Encoder{i}_w0", mann.encoders[i].W[0].shape)
    print(f"Encoder{i}_b0", mann.encoders[i].b[0]._variable)
    # mann.encoders[i].W[0] = tf.Variable(bindata[f'Encoder{i}_w0'].view(mann.encoders[i].W[0].shape))
    # mann.encoders[i].b[0] = tf.Variable(bindata[f'Encoder{i}_b0'].view(mann.encoders[i].b[0].shape))
for i in range(3):
    print(f"DensResNet{i}_w0", mann.DensResNet[i].W[0].shape)
    print(f"DensResNet{i}_b0", mann.DensResNet[i].b[0].shape)
    # mann.DensResNet[i].W[0] = tf.Variable(bindata[f'DensResNet{i}_w0'].view(mann.DensResNet[i].W[0].shape))
    # mann.DensResNet[i].b[0] = tf.Variable(bindata[f'DensResNet{i}_b0'].view(mann.DensResNet[i].b[0].shape)
print(mann.decoder.W[0].shape)
# mann.decoder.W[0] = tf.Variable(bindata['Decoder_w0'])
# mann.decoder.b[0] = tf.Variable(bindata['Decoder_b0'])

# for i in range(1, 4):
#     mann.DensResNet[i].W[0] = tf.Variable(bindata['Decoder_w'+str(i)])
#     mann.decoder.b[i] = tf.Variable(bindata['Decoder_b'+str(i)])

Encoder0_w0 (192, 512)
Encoder0_b0 Tensor("encoder0b0:0", shape=(512,), dtype=float32_ref)
Encoder1_w0 (336, 512)
Encoder1_b0 Tensor("encoder1b0:0", shape=(512,), dtype=float32_ref)
DensResNet0_w0 (1536, 1536)
DensResNet0_b0 (1536,)
DensResNet1_w0 (1536, 1536)
DensResNet1_b0 (1536,)
DensResNet2_w0 (1536, 1536)
DensResNet2_b0 (1536,)
(1536, 214)


In [8]:
name_model = "ManipNet"
keep_prob_main = 0.7
keep_prob_encoders = [0.7, 0.7, 0.7]
learning_rate = 0.0001

GPU_Occupancy = 0.9
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_Occupancy)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

mann.Train(
    sess,
    # Model name for Saving
    name_model,
    # Flag/Path of test data
    path_test=test_path,
    flag_save_bin = True,
    step_save=100, max_save=3,
    # HyperParameters
    keep_prob_main=keep_prob_main, 
    batch_size=32, 
    epoch=500, 
    learning_rate_ini=learning_rate,
    keep_prob_encoders=keep_prob_encoders
    )

2024-03-13 23:53:31.768299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.6705
pciBusID: 0000:65:00.0
2024-03-13 23:53:31.768639: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2024-03-13 23:53:31.768808: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory
2024-03-13 23:53:31.768960: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcufft.so.10.0'; dlerror: libcufft.so.10.0: cannot open shared object file: No such file or directory
2024-03-13 23:53:31.769102: W tensorflow/stream_executor/platform/default/dso_loader.cc:55]

Training information
Total Batch -> 316
--------------------
Input X -> 2350
Output Y -> 214
--------------------
Main Keep Prob -> 0.7
Encoders Keep Prob -> [0.7, 0.7, 0.7]
TestData is Processed


Epoch: 0001 Training Loss = 7.037237725
Epoch: 0001 Testing Loss = 0.699002019


KeyboardInterrupt: 